In [15]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scienceplots

qml.drawer.use_style(style='sketch')

## Description of Quantum matrix multiplication algorithm

### Classical matrix multiplication

**Problem**: Given positive definite matrix $A$ and the vector $x$, find the vector $b$ such that $b = Ax$.

1. Find eigenvalues and eigenvectors of $A$.
   $$
   Av_r = \lambda_r v_r
   $$

2. Express $x$ in terms of eigenvectors of $A$.
   $$
   x = \sum_{r=1}^{R} (v_r^T x) v_r
   $$

3. Multiply $A$ and $x$.
   $$
   Ax = \sum_{r=1}^{R} \lambda_r (v_r^T x) v_r
   $$

### Quantum matrix multiplication

1. Prepare the quantum state $|\psi_x\rangle$ corresponding to the vector $x$ via amplitude encoding.
   $$
   |\psi_x\rangle = \sum_{r=1}^{R} x_i |i\rangle
   $$

2. Encoding the matrix $A$ into a Hamiltonian $H_A$ using Hamiltonian encoding. (This step allows us to use non-hermitian $A$)
   $$
   H_A = \begin{pmatrix} 0 & A \\ A^\dagger & 0 \end{pmatrix} 
   $$
   Then the problem is changed to solve $H_A y = \begin{pmatrix} b \\ 0 \end{pmatrix}$ to obtain $y = \begin{pmatrix} 0 \\ x \end{pmatrix}$.

3. Construct the unitary operator $U = e^{2\pi i H_A}$ using _Hamiltonian simulation_.

4. Prepare ancilla qubits and apply controlled sequence of $U$ to use the quantum phase estimation algorithm.
   $$
   \frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^n-1} U^k|k\rangle |\psi_x\rangle = \frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^n-1} |k\rangle \sum_{r=1}^{R} e^{2\pi i k \lambda_r} \langle \psi_{v_r} | \psi_x \rangle |\psi_{v_r}\rangle
   $$

5. Apply QPE to register $|k\rangle$ to obtain the eigenvalues $\lambda_r$.
   $$
   \frac{1}{\sqrt{2^n}} \sum_{r=1}^{R} \alpha_{k|r} \langle \psi_{v_r} | \psi_x \rangle |k\rangle |\psi_{v_r}\rangle
   $$
   We can find $|\lambda_r\rangle$ by measuring the register $|k\rangle$.
   $$
   \sum_{r=1}^R \langle \psi_{v_r} | \psi_x \rangle |\lambda_r\rangle |\psi_{v_r}\rangle
   $$

6. Use another ancilla qubit to encoding the eigenvalues into the amplitudes.
   To implement this, use controlled sequence of rotation gates.
   $$
   \sum_{r=1}^R \langle \psi_{v_r} | \psi_x \rangle |\lambda_r\rangle |\psi_{v_r}\rangle |0\rangle \rightarrow
   \sum_{r=1}^R \langle \psi_{v_r} | \psi_x \rangle |\lambda_r\rangle |\psi_{v_r}\rangle \left( \sqrt{1-\lambda_r^2} |0\rangle + \lambda_r |1\rangle \right)
   $$
   > _How to implement the controlled rotation gates?_
   >
   > Our goal is to find $\theta$ to input into the rotation gate $R_y(\theta)$.
   > $$
   > \begin{aligned}
   >  R Y(\theta)|0\rangle_{\mathrm{a}} & =\left(\begin{array}{cc}
   >  \cos \left(\frac{\theta}{2}\right) & -\sin \left(\frac{\theta}{2}\right) \\
   >  \sin \left(\frac{\theta}{2}\right) & \cos \left(\frac{\theta}{2}\right)
   >  \end{array}\right)\left(\begin{array}{l}
   >  1 \\
   >  0
   >  \end{array}\right) \\
   >  & =\cos \left(\frac{\theta}{2}\right)|0\rangle_{\mathrm{a}}+\sin \left(\frac{\theta}{2}\right)|1\rangle_{\mathrm{a}}
   >  \end{aligned}
   > $$
   > Our desired $\theta$ is $2\arcsin\left(\dfrac{1}{\lambda_r}\right)$. From QPE, we can find the binary representation of $\lambda_r$ and then calculate $\theta$.

7. Do branch selection: measure the ancilla qubit and continue only if we get $|1\rangle$.
   $$
   \sum_{r=1}^R \lambda_r \langle \psi_{v_r} | \psi_x \rangle |\lambda_r\rangle |\psi_{v_r}\rangle |1\rangle \rightarrow \sum_{r=1}^R \lambda_r \langle \psi_{v_r} | \psi_x \rangle |\psi_{v_r}\rangle
   $$
   This result corresponds to a normalized version of $Ax$ in amplitude encoding.

## Prepare operations for QMM

In [20]:
def unitary_matrix_from_hermitian(A):
    eigenvalues, eigenvectors = np.linalg.eigh(A)
    D = np.diag(np.exp(2j * np.pi * eigenvalues))
    U = np.dot(eigenvectors, np.dot(D, eigenvectors.conj().T))
    return U

def hamiltonian_encoding(A):
    I_first = np.array([[0, 1], [0, 0]])
    I_second = np.array([[0, 0], [1, 0]])
    H = np.kron(I_first, A) + np.kron(I_second, A.conj().T)
    return H

In [21]:
A = np.array([[1, 2], [1, 3]])
H = hamiltonian_encoding(A)
H

tensor([[0, 0, 1, 2],
        [0, 0, 1, 3],
        [1, 1, 0, 0],
        [2, 3, 0, 0]], requires_grad=True)

In [22]:
U = unitary_matrix_from_hermitian(H)
U

tensor([[ 1.81562209e-01-1.91513472e-15j,
          3.35854164e-01-2.33146835e-15j,
         -1.36002321e-15+6.04870071e-01j,
         -2.66453526e-15-6.98834274e-01j],
        [ 3.35854164e-01-2.30371278e-15j,
          4.21458040e-01-2.88657986e-15j,
         -1.09634524e-15-7.58397369e-01j,
         -4.27435864e-15-3.66617691e-01j],
        [-1.36002321e-15+6.04870071e-01j,
         -1.09634524e-15-7.58397369e-01j,
          3.76247104e-02-1.07552856e-15j,
          2.39895831e-01-1.85962357e-15j],
        [-2.69229083e-15-6.98834274e-01j,
         -4.32986980e-15-3.66617691e-01j,
          2.39895831e-01-1.85962357e-15j,
          5.65395539e-01-3.94129174e-15j]], requires_grad=True)

In [23]:
# is unitary?
np.allclose(U @ U.conj().T, np.identity(4))

True

In [24]:
# QPE
def convert_theta(theta):
    # to input PhaseShift gate
    return 2 * theta * np.pi

def gen_qpe_circuit(n:int, dev):
    @qml.qnode(dev)
    def circuit(theta:float):
        # For an eigenstate |psi> of U (PhaseShift gate)
        qml.PauliX(wires=n)

        # Apply Hadamard gate to the first n qubits
        for i in range(n):
            qml.Hadamard(wires=i)
        
        # Apply controlled unitary operations
        #for i in reversed(range(n)):
            #qml.ControlledPhaseShift(convert_theta(theta) * 2**(n-1-i), wires=[i, n])
        qml.ControlledSequence(qml.PhaseShift(convert_theta(theta), wires=n), control=range(n))
        
        # Apply inverse QFT
        qml.adjoint(qml.QFT)(wires=range(n))

        return qml.probs(wires=range(n))

    return circuit

In [ ]:
def gen_qmm(n: int, A):
    U = unitary_matrix_from_hermitian(hamiltonian_encoding(A))
    dev = qml.device("default.qubit", wires=n+2) # ancilla, c-register, encoding x
    @qml.qnode(dev)
    def circuit(x):
        # Encode the input vector x into the amplitudes of the state (here, only 2d vectors)
        qml.AmplitudeEmbedding(features=x, wires=n+1, normalize=True)

        # Apply Hadamard gate to the first n qubits
        for i in range(1, n+1):
            qml.Hadamard(wires=i)

        # Apply controlled unitary operations
        qml.ControlledSequence(qml.QubitUnitary(U, wires=n+1), control=range(1,n+1))

        # Apply inverse QFT
        qml.adjoint(qml.QFT)(wires=range(1,n+1))
        # Then the state is |lambda> with highest probability

        # Apply controlled rotations for |lambda> -> |lambda> (sqrt(1-lambda^2) |0> + lambda |1>)
